## Development of rejection samplers 

##### Author: [Nicholas B](mailto:nicholas92457@gchq.gov.uk)
This notebook establishes the rejection samplers used in library and recorded in the paper.

The objective is to construct a rejection sampler for the following density:

$$f(x) = \frac{1}{N}e^{-\frac{x^2}{2}}\sum_{i}\phi_i(x)^2$$

where $\phi_i$ are multivariate Hermite polynomials on $\mathbb{R}^d$ with multi-index $i = (i_1,\ldots, i_d)$. The sum is over all of the first $N$ multi-indices, where the multi-indices are given the natural ordering by max-degree first and lexicographic second. We restrict to the case where $N = n^d$. We have $$\phi_i(x) = \prod_{k=1}^d \psi_{i_k}(x_k)$$
where $\psi_i$ are standard probabilist's normalised Hermite polynomials. We can now write 
$$ f(x) = \prod_{k=1}^d\left(\frac{1}{n}e^{-\frac{x_k^2}{2}}\sum_{i=1}^n\psi_i(x_k)^2\right)\equiv \prod_{k=1}^d f_k(x_k).$$

As $n\rightarrow \infty$ the densities $f_k$ approach the wigner semi-circle, so we will construct a rejection sampler based on the semi-circle. However, we must use a mixture of the semi-circle with some heavy-tailed distribution, as $f_k$ is supported on $\mathbb{R}$ with sub-Gaussian tails.

In [ ]:
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import eval_hermitenorm, eval_hermite, factorial
from scipy import optimize, stats
from tqdm import tqdm

In [ ]:
def semi_circle(x):
    """Wigner semi-circle."""
    z = np.maximum(0, 4 - x**2)
    return np.sqrt(z)/(2*np.pi)

def finite_n_semi_circle_density(x, n):
    """The density we are targetting."""
    _x = x
    prefactor = 1 / n
    hermite = sum(eval_hermite(ind, _x / np.sqrt(2))**2 / (2**ind * factorial(ind)) for ind in range(n))
    return prefactor * hermite * stats.norm.pdf(_x)

In [ ]:
N = 50
x = np.linspace(-np.sqrt(4*N)-1, np.sqrt(4*N)+1, 1000)
plt.plot(x, finite_n_semi_circle_density(x, N));
plt.plot(x, semi_circle(x/np.sqrt(N))/np.sqrt(N));

We need to sample from the semi-cirle law. It may seem natural to sample Wigner matrix eigenvalues (e.g. from a GOE which is simple to sample), but we found empirically this to be much slower, so we implement a rejection sampler. The proposal distribution is just a symmetric mixture of two Gaussians and we hand-tune a good rejection bound.

In [ ]:
semi_circle_rejection_bound = .71/.5
def gmm_semicircle_proposal(x):
    return (stats.norm.pdf(x, loc=-1, scale=0.8) + stats.norm.pdf(x, loc=1, scale=0.8))/2

x = np.linspace(-3, 3, 1000);
plt.plot(x, semi_circle_rejection_bound * gmm_semicircle_proposal(x), label="$q(x) * M$")
plt.plot(x, semi_circle(x), label="p(x)");
plt.legend();

In [ ]:
def sample_semicircle_rejection():
    for _ in range(1000):
        mean = -1 if np.random.randint(2)==0 else 1
        proposal_point = stats.norm.rvs(loc=mean, scale=.8)
        ratio = semi_circle(proposal_point)/((stats.norm.pdf(proposal_point, loc=-1, scale=0.8) + stats.norm.pdf(proposal_point, loc=1, scale=0.8))/2)
        u = np.random.uniform()
        if u * semi_circle_rejection_bound < ratio:
            return proposal_point

In [ ]:
plt.hist([sample_semicircle_rejection() for _ in range(2000)], bins=20, density=True);
plt.plot(x, semi_circle(x));

In [ ]:
DEGREES_FREEDOM = 10
MIXTURE_PROB = lambda n: 0.4# 0.5/(n-1)

def semi_circle_student_mixture_proposal_density(z, mixture_prob, nu=DEGREES_FREEDOM):
    """
    Computes the density of a Student-t and semi-circle mixture.

    :param float,np.ndarray[float] z: The point(s) at which to evaluate the density.
    :param float mixture_prob: The probability of the Student-t component.

    :rtype: np.ndarray[float]
    """
    return semi_circle(z) * (1-mixture_prob) + stats.t.pdf(z, nu) * mixture_prob


def sample_semi_circle_student_mixture(p, n=DEGREES_FREEDOM):
    """
    Sample from a mixture of semi-circle random variable and a Student-t.
    """
    if np.random.binomial(1, p) == 1:
        return stats.t.rvs(nu)
    else:
        return sample_semicircle_rejection()

Here we have built what looks like a decent semicircle-Student mixture to act as proposal distribution for $f_k(.)$. We now want to optimise the rejection bound.

The following cell tries various mixture probabilities and degrees of freedom for each $n$ and estimates the best acceptance probability that can be achieved. Thus we can find near optimal values of mixture probability and degrees of freedom for each $n$. This cell will run in under 10 minutes on a decent machine capable of the degree of parallelism employed below.

We load in the results of running the same code on a bigger machine.

In [ ]:
online = False

if online:
    ns = np.arange(2, 60)
    ps = np.linspace(1e-2, .5, 10)
    nus = np.linspace(1, 30, 10).astype(int)
    n_trials = 1000

    def n_optimiser(nn):
        success_rates = []
        x = np.linspace(-np.sqrt(4*nn)-10, np.sqrt(4*nn)+10, 5000)
        for p in ps:
            for nu in nus:
                M = np.max(finite_n_semi_circle_density(x, nn)/(semi_circle_student_mixture_proposal_density(x/np.sqrt(nn), p, nu)/np.sqrt(nn))) * 1.025
                n_success = 0
                for _ in range(n_trials):
                    proposal = sample_semi_circle_student_mixture(p, nu) * np.sqrt(nn)
                    ratio = finite_n_semi_circle_density(proposal, nn)/( semi_circle_student_mixture_proposal_density(proposal/np.sqrt(nn), p, nu)/np.sqrt(nn))
                    u = np.random.uniform()
                    if u * M < ratio:
                        n_success += 1
                success_rates.append((p, nu, n_success/n_trials))
        return success_rates

    results = Parallel(n_jobs=50)(delayed(n_optimiser)(n) for n in ns)
    results = [[[float(rr) for rr in r] for r in result] for result in results]
else:
    import json
    with open("optimise_rejection.json") as fin:
        ns, results = json.load(fin)

In [ ]:
best_results = [max(r, key=lambda x: x[2]) for r in results]

We plot the best mixture probability and degrees of freedom by $n$. Best meaning the acceptance ratio.

In [ ]:
plt.plot(ns, [b[0] for b in best_results]);
plt.xlabel("n")
plt.ylabel("p")
plt.show()

In [ ]:
plt.plot(ns, [b[1] for b in best_results]);
plt.xlabel("n")
plt.ylabel("degrees freedom")
plt.show()

In [ ]:
DEGREES_FREEDOM = 10
MIXTURE_PROB = lambda n: 0.4# 0.5/(n-1)


Degrees of freedom seem unimportant, so we fix at 10 for all $n$. We will fit $p_n$. 

In [ ]:
POWERS = [0, -1, -.5, -.25]

optimal_ps = np.array([b[0] for b in best_results])
ns = np.array(ns).astype(float)

def fit_loss(coeffs):
    features = np.stack([ns**power for power in POWERS], -1)
    return np.sum((features @ coeffs - (optimal_ps + 0.03))**2)

p_optimal_coefficients = optimize.minimize(fit_loss, np.ones(len(POWERS))).x

def mixture_probability(nn):
    if isinstance(nn, np.ndarray):
        nn = nn.astype(float)
    else:
        nn = float(nn)
    return np.stack([np.power(nn, power) for power in POWERS], -1) @ p_optimal_coefficients

plt.plot(ns, optimal_ps, label="empirical");
plt.xlabel("$n$")
plt.ylabel("mixture probability")
plt.plot(ns, mixture_probability(ns.astype(float)), label="fit");
plt.legend()
plt.savefig("../paper/figures/optimal_mixture.pdf")
print(p_optimal_coefficients)

In [ ]:
n = np.arange(2, 70)
M =[]
for nn in n:
    x = np.linspace(-np.sqrt(4*nn)-10, np.sqrt(4*nn)+10, 5000)
    M.append(np.max(finite_n_semi_circle_density(x, nn)/(semi_circle_student_mixture_proposal_density(x/np.sqrt(nn), mixture_probability(nn))/np.sqrt(nn))))
plt.plot(n, M);
rejection_bounds = np.array(M)

In [ ]:
POWERS = [0, -1, -.5, -.25]

def fit_loss(coeffs):
    features = np.stack([n.astype(float)**power for power in POWERS], -1)
    return np.sum((features @ coeffs - (rejection_bounds + 0.03))**2)

m_optimal_coefficients = optimize.minimize(fit_loss, np.ones(len(POWERS))).x

def rejection_bound(nn):
    if isinstance(nn, np.ndarray):
        nn = nn.astype(float)
    else:
        nn = float(n)
    return 0.03 + np.stack([np.power(nn, power) for power in POWERS], -1) @ m_optimal_coefficients

plt.plot(n, rejection_bounds, label="empirical");
plt.plot(n, rejection_bound(n.astype(float)), label="fit");
plt.xlabel("$n$")
plt.ylabel("rejection bound")
plt.legend()
plt.savefig("../paper/figures/optimal_bound.pdf")
print(m_optimal_coefficients)

Check it generalises.

In [ ]:
n = np.arange(2, 100)
M =[]
for nn in n:
    x = np.linspace(-np.sqrt(4*nn)-10, np.sqrt(4*nn)+10, 5000)
    M.append(np.max(finite_n_semi_circle_density(x, nn)/(semi_circle_student_mixture_proposal_density(x/np.sqrt(nn), mixture_probability(nn))/np.sqrt(nn))))
plt.plot(n, M);
plt.plot(n, rejection_bound(n.astype(float)));

In [ ]:
def rejection_sample_finite_n_density(n):
    bound  = rejection_bound(n)
    for _ in range(1000):
        proposal = sample_semi_circle_student_mixture(mixture_probability(n)) * np.sqrt(n)
        ratio = finite_n_semi_circle_density(proposal, n)/( semi_circle_student_mixture_proposal_density(proposal/np.sqrt(n), mixture_probability(n))/np.sqrt(n) )
        u = np.random.uniform()
        if u * bound < ratio:
            return proposal

In [ ]:
n = 10
plt.hist([rejection_sample_finite_n_density(n) for _ in range(10000)], bins=50, density=True, label='samples')
x = np.linspace(-np.sqrt(4*n)-1, np.sqrt(4*n)+1, 1000)
plt.plot(x, finite_n_semi_circle_density(x, n), label="density", linewidth=2);
plt.legend()
plt.savefig(f"../paper/figures/sample_n{n}.pdf")

In [ ]:
N = 10
x = np.linspace(-np.sqrt(4*N)-1, np.sqrt(4*N)+1, 1000)
plt.plot(x, finite_n_semi_circle_density(x, N), label="$\\rho(x)$")
proposal_density = semi_circle_student_mixture_proposal_density(x/np.sqrt(N), mixture_probability(N))/np.sqrt(N)
plt.plot(x, rejection_bound(N) * proposal_density, label="$Mf(x)$")
plt.legend();
plt.xlabel("$x$")
plt.savefig(f"../paper/figures/densities_n{N}.pdf")

In [ ]:
n_trials = 10000
ns = np.arange(2, 60)
accept_ratios = []

for n in tqdm(ns):
    accepts = 0
    bound  = rejection_bound(n)
    for _ in range(10000):
        proposal = sample_semi_circle_student_mixture(mixture_probability(n)) * np.sqrt(n)
        ratio = finite_n_semi_circle_density(proposal, n)/( semi_circle_student_mixture_proposal_density(proposal/np.sqrt(n), mixture_probability(n))/np.sqrt(n))
        u = np.random.uniform()
        if u * bound < ratio:
            accepts += 1
    accept_ratios.append(accepts/n_trials)

In [ ]:
plt.plot(ns, accept_ratios)
plt.ylabel("accept probability")
plt.xlabel("$n$")
plt.savefig("../paper/figures/accept_prob.pdf")

## General $N$ sampling

Here we produce estimates of the acceptance probabilities for a sampler from the multivariate density

$$f(x) = \frac{1}{N}e^{-\frac{x^2}{2}}\sum_{i=1}^N\phi_i(x)^2$$

for any $N$. We choose the least $n$ such that $n^d \geq N$ and define $m = n^d - N$. Then define the proposal density 

$$q(x) = \frac{1}{n^d}e^{-\frac{x^2}{2}}\sum_{i=1}^{n^d}\phi_i(x)^2$$

and we have the bound $$ \frac{f(x)}{q(x)} \leq \frac{n^d}{n^d - m}.$$

Sampling from $q$ amounts to simply sampling $d$ i.i.d. entries from the univariate $$  \frac{1}{n}e^{-\frac{x_j^2}{2}}\sum_{i=1}^n\psi_i(x_j)^2$$ as above. 

Thus we have constructed all we need for a rejection sampler.

In [ ]:
import sys

sys.path.append("..")
from semi_circle_cython_backend import sample_finite_n_semi_circle
from hermitedpp.multivariate_hermite_factor_ope import MultivariateHermiteFactorOPE

In [ ]:
def count_accepts(dpp_instance, nb_trials_max=1000):
    success = 0
    for _ in range(nb_trials_max):
        proposal_point = sample_finite_n_semi_circle(dpp_instance.single_factor_n, n_samples=dpp_instance.dim)
        phi_x = dpp_instance.eval_multiD_polynomials(proposal_point, all=True)
        target_density = phi_x[:, :dpp_instance.n_points] @ phi_x[:, :dpp_instance.n_points].T
        proposal_density = phi_x @ phi_x.T
        uniform = np.random.uniform()
        if uniform * dpp_instance.nearest_factor_rejection_bound < target_density / proposal_density:
            success += 1
    return success / nb_trials_max

In [ ]:
n_trials = 1000
dim = 2
ns = np.linspace(26, 1000, 30).astype(int)
accept_ratios = []

for n in tqdm(ns):
    dpp = MultivariateHermiteFactorOPE(n, dim)
    accept_ratios.append(count_accepts(dpp))


plt.plot(ns, accept_ratios, "x-", label="observed")
plt.ylabel("accept probability")
plt.xlabel("$N$")
plt.savefig(f"../paper/figures/accept_prob_bootstrap_d{dim}.pdf")

In [ ]:
n_trials = 1000
dim = 3
ns = np.linspace(26, 1000, 30).astype(int)
accept_ratios = []

for n in tqdm(ns):
    dpp = MultivariateHermiteFactorOPE(n, dim)
    accept_ratios.append(count_accepts(dpp))

plt.plot(ns, accept_ratios, "x-", label="observed")
plt.ylabel("accept probability")
plt.xlabel("$N$")
plt.savefig(f"../paper/figures/accept_prob_bootstrap_d{dim}.pdf")

In [ ]:
5**4

In [ ]:
n_trials = 1000
dim = 4
ns = np.linspace(26, 1000, 30).astype(int)
accept_ratios = []

for n in tqdm(ns):
    dpp = MultivariateHermiteFactorOPE(n, dim)
    accept_ratios.append(count_accepts(dpp))

plt.plot(ns, accept_ratios, "x-", label="observed")
plt.ylabel("accept probability")
plt.xlabel("$N$")
plt.savefig(f"../paper/figures/accept_prob_bootstrap_d{dim}.pdf")